# Find a Feasible Mobile Manipulation (from Real Data)

In [1]:
%matplotlib widget

import numpy as np
import feasibility_map as fmap

## 1. Offline

### 1.0. Utils

### 1.1. Configuration

1. `Rsize`: the radius of the robot-base
2. `M`: X-Y-Cr manipulability map

```python
raw:
    [[Cr x y manip],
     [Cr x y manip], ...]
layers:
    [deg]: [[x, ...],
            [y, ...],
            [manip, ...]]
```

In [2]:
Rsize = 0.3

- npy data
    - Cr (radian)
    - x (m)
    - y (m)
    - manip (no unit)

In [3]:
# In this file, Cr is radians.
M = np.load('../config/robocare_right_arm_sample.npy')
# So, convert it to degrees.
M[:,0] = np.round(np.degrees(M[:,0]), decimals=3)
print(M.shape)
print(M)

(893, 4)
[[ 0.00000e+00  1.00000e-01 -4.40000e-01  1.51625e-02]
 [ 1.00000e+01  1.00000e-01 -4.40000e-01  1.51625e-02]
 [ 2.00000e+01  1.00000e-01 -4.40000e-01  1.51625e-02]
 ...
 [ 6.00000e+01  2.80000e-01 -8.00000e-02  8.97977e-03]
 [ 7.00000e+01  2.80000e-01 -8.00000e-02  8.97977e-03]
 [ 8.00000e+01  2.80000e-01 -8.00000e-02  8.97977e-03]]


### 1.2. Preparation

1. Convert the `M` to a feasibility map `F` (helical shape).

Cr, x, y, manipulability

In [4]:
F = fmap.FeasibilityMap(M, Rsize, is_jupyter=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(3, 99)
(3, 99)
(893, 4)


## 2. Online

### 2.1. Input

Known
1. `self.feasi_layers`: feasibility map
2. `self.robot_radius`: robot base (circle radius)

---
Input

1. `Pt`: Position of the target object (relative to the robot’s current pose) => `(x, y)`
2. `Obs`: Area list of ground obstacles => `[CollisionModel, ...]`
3. `Cr`: Constraints on the approach angle (relative to the robot heading) (-90 ~ 90) => `(min, max)`
4. `Ct`: Constraints on the approach angle (relative to the target heading) (-180 ~ 180) => `(min, max)`

In [5]:
from collision import CollisionBox

Pt = (0.0, 0.0)
Obs = [CollisionBox((0.2, 0.0), np.radians(0.0), 0.45, 1.0)]
# Cr = (30, 30)  # min, max
Cr = (0, 0)
Ct = (-np.radians(180), np.radians(180)) # min, max

### 2.2. Process

1. Cut the range of `Cr` from `F` and set it to `Fcut`.
2. ~~[SKIP] Scan the maximum points for the radius and angle by each circle in `Fcut`.~~
3. And extract only the maximum as a `Fmax`.
4. Wipe the `Fmax` in the range of `Ct`.
5. Remove all obstacle areas from `Fwiped` with the offset of `Rsize`.

In [6]:
F.calc(Pt, Obs, Cr, Ct, (0.05, 0.5, 0.02))

Cut the range of `Cr` from `Fraw` and set it to `Fcut`.
min Cr:  0.0
max Cr:  0.0


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

And extract only the maximum as a `Fmax`.
sq_sections.shape:  (23,)
Fmax.shape:  (19, 4)
Fmax:
 [[ 0.        -0.1        0.02       0.0430307]
 [ 0.        -0.1        0.05       0.0404036]
 [ 0.        -0.13       0.02       0.0420638]
 [ 0.        -0.16       0.05       0.0407139]
 [ 0.        -0.16       0.08       0.0397802]
 [ 0.        -0.19       0.08       0.0389292]
 [ 0.        -0.19       0.11       0.038651 ]
 [ 0.        -0.19       0.14       0.0377761]
 [ 0.        -0.19       0.17       0.0372181]
 [ 0.        -0.1        0.26       0.0408668]
 [ 0.        -0.1        0.29       0.0461509]
 [ 0.        -0.13       0.29       0.048317 ]
 [ 0.        -0.13       0.32       0.048756 ]
 [ 0.        -0.16       0.32       0.0478867]
 [ 0.        -0.13       0.35       0.0448253]
 [ 0.        -0.1        0.38       0.0405654]
 [ 0.        -0.1        0.41       0.0316079]
 [ 0.        -0.13       0.41       0.0240168]
 [ 0.        -0.1        0.44       0.0151625]]
When Ct ==

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Wipe the `Fmax` in the range of `Ct`. => `Fwiped`
min Ct:  -180.0
max Ct:  180.0
Fwiped.shape:  (1652, 5)
Interval is  0.02  [m].


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Remove all obstacle areas from `Fwiped` with the offset of `Rsize`. => `Fclean`
Fclean.shape:  (1652, 5)
Fclean.shape:  (163, 5)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 2.3. Output

1. Candidate poses (sorted in descending order of manipulability)

In [7]:
candidates = F.get_candidates()

Fclean[:10]:
 [[ 4.97649627e+01  0.00000000e+00 -3.28258572e-01  1.07453731e-01
   4.87559997e-02]
 [ 5.30826269e+01  0.00000000e+00 -3.33926960e-01  8.82767228e-02
   4.87559997e-02]
 [ 5.64002911e+01  0.00000000e+00 -3.38476039e-01  6.88038151e-02
   4.87559997e-02]
 [ 5.97179552e+01  0.00000000e+00 -3.41890562e-01  4.91002800e-02
   4.87559997e-02]
 [ 6.30356194e+01  0.00000000e+00 -3.44159084e-01  2.92321630e-02
   4.87559997e-02]
 [ 6.63532836e+01  0.00000000e+00 -3.45273999e-01  9.26606108e-03
   4.87559997e-02]
 [ 6.96709478e+01  0.00000000e+00 -3.45231571e-01 -1.07311003e-02
   4.87559997e-02]
 [ 7.29886119e+01  0.00000000e+00 -3.44031942e-01 -3.06922915e-02
   4.87559997e-02]
 [ 7.63062761e+01  0.00000000e+00 -3.41679134e-01 -5.05506034e-02
   4.87559997e-02]
 [ 7.96239403e+01  0.00000000e+00 -3.38181033e-01 -7.02394721e-02
   4.87559997e-02]]
Fclean[:10] (sorted):
 [[ 4.97649627e+01  0.00000000e+00 -3.28258572e-01  1.07453731e-01
   4.87559997e-02]
 [ 8.62592687e+01  0.000000

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …